<img align="center" width="12%" style="padding-right:10px;" src="Images/Ccom.png">

# Lab 4.0: Positioning Data<a href="https://piazza.com/class/jzvaaav18cf2j7"><img src="Images/help.png"  title="Ask questions on Piazza.com" align="right" width="10%" alt="Piazza.com\"></a><br><br> 

In this Notebook you will update the Position class from the Integrated Seabed Mapping Systems course Lab A 
assignment. The updates consists of adding a read method to parse the positioning data contained within HYPACK *.RAW* files.

For now the only positioning data that you will extract is that contained withing the **GGA** records

You have not yet encountered irregular ascii files i.e., files that do not have a predictable structure, this is a gentle introduction into it



___
This time we will create a copy of the Position class, so that it does not interfere with your work in the 874 class, normally we would just use the same class definition

<img align="left" width="6%" style="padding-right:10px;" src="Images/test.png"><br>
Copy your position class to the mycode section for ESCI_872_Clean - I will show in class how to do this
<br><br>



In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np

sys.path.append(os.getcwd())  # add the current folder to the list of paths where Python looks for modules 

Add the following code to Position class in the file position.py


    def read_hypack_raw_file(self, fullpath):
        
        # This function will currently only function provided that there are GGA sentences in the records.
        # You may update the function to include other positioning messages as well, but this is 
        # outside the scope of the class
                        
        # Check the File's existence
        if os.path.exists(fullpath):
            self.data_path = fullpath
            print('Opening GNSS data file:' + fullpath)
        else:  # Raise a meaningful error
            raise RuntimeError('Unable to locate the input file' + fullpath)
            
        # Open, read and close the file
        hypack_file = open(fullpath)
        hypack_content = hypack_file.read()
        hypack_file.close    

        # Split the file in lines
        hypack_records = hypack_content.splitlines()
        
        # Go through the header lines to find the date of the survey (not contained in the GGA records)
        
        lines_parsed=0
        for hypack_record in hypack_records:
            
            # Check for the time and date
            
            if hypack_record[:3].lower() == "tnd":
                hypack_datetime=datetime.strptime(hypack_record[4:23], "%H:%M:%S %m/%d/%Y")
                
                print("HYPACK RAW Header start time and date: " + hypack_datetime.ctime())
                
            # Keep track of the lines parsed
            lines_parsed+=1

            # Stop going through the records if the record starts with the string eoh (End Of Header)
            if hypack_record[:3].lower() == "eoh":
                break         
        
        # We are at the end of the header - start looking for the first GGA record and compare its time 
        # to the TND record
        # This is so that we can set the correct date
        
        # Keep track of the number of GGA records found
        
        num_gga_recs=0
        
        for hypack_record in hypack_records[lines_parsed:]:

            if hypack_record[19:22] == "GGA":
                gga_data=hypack_record.split()[3]
                gga_data=gga_data.split(',')
                
                # Determine the time of day from both the header and the GGA string
     
                gga_timedelta=timedelta(hours=int(gga_data[1][0:2]), \
                                        minutes = int(gga_data[1][2:4]), \
                                        seconds = int(gga_data[1][4:6]))
            
                hypack_timedelta=timedelta(hours = hypack_datetime.hour, \
                                           minutes = hypack_datetime.minute, \
                                           seconds = hypack_datetime.second)
            
                # If the hours are not the same we need to believe one or the other, which one?
                # We know from experience that the GGA time from Trimble receivers is indeed utc, 
                # whereas we also know that in older version of HYPACK the timestamp depended on the CPU time
                
                # Why do we care? We may end up with the timing for the wrong day if we are not careful! 
                # To know the correct date we need to take the date and time from the header and adjust it to UTC,
                # After that we know the time. If then the time difference between any succeeding GGA records 
                # is negative 
                # we know that we have just gone into a new day and need to update the datetime. 
                # Calculate the time difference in microseconds

                # Convert the HYPACK datetime to represent UTC time. We will make the ASSUMPTION that the 
                # header was created just before the first GGA record is received - this is _NOT_ fool 
                # proof (why?) 
                
                # On Piazza discuss a method to make this more robust by taking the time difference in minutes and 
                # seconds between the header time and the first GGA record time into account
                                
                hypack_datetime = hypack_datetime + \
                    timedelta( hours = round((gga_timedelta - hypack_timedelta).total_seconds() / 3600))
                
                print("HYPACK RAW Header start time and date in UTC: " + hypack_datetime.ctime())
                
                gga_datetime = hypack_datetime
                
                break
                
            # Keep track of where we are
                
            lines_parsed+=1
            
                
        # We are in the data file at the first GGA record - we know the date
  
        prev_gga_time = gga_timedelta
   
        
        for hypack_record in hypack_records[lines_parsed:]:
            if hypack_record[19:22] == "GGA":
                
                # Update the number of GGA records found
                
                num_gga_recs += 1
                
                # Get the GGA string and tokenize it
                
                gga_data=hypack_record.split()[3]
                gga_data=gga_data.split(',')
                
                # Determine the time of day from both the header and the GGA string
     
                gga_timedelta=timedelta(hours=int(gga_data[1][0:2]), \
                                        minutes = int(gga_data[1][2:4]), \
                                        seconds = int(gga_data[1][4:6]))
            
                # Check whether we rolled into a new day - we assume time is always increasing
                
                if gga_timedelta < prev_gga_time:
                    # We have reached the next day - update the date
                    
                    gga_datetime += timedelta(days=1)
                    
                    print( "Passed midnight, updating date to: "+str(gga_datetime.date()))
                    
                # Create the time
                
                time=gga_datetime.replace(hour=int(gga_data[1][0:2]), \
                                        minute = int(gga_data[1][2:4]), \
                                        second = int(gga_data[1][4:6]))

                # Add the time object to the list of times

                # Insert code here ***
                
                # Parse the latitude

                if gga_data[3].lower() == "n":
                    self.latitudes.append( float(gga_data[2][0:2])+float(gga_data[2][3:])/60.)
                else:
                    # Insert code here ***               

                # Parse the longitudes - NOTE that longitude used 3 digits before the decimal

                if gga_data[5].lower == "w":
                    # Insert code here ***
                else:
                    # Insert code here ***               

                # Parse the GNSS Quality indicator
                
                # Insert code here ***

                # Parse the number of GNSS satellites used for the solution
                
                # Insert code here ***

                # Parse the HDOP Quality indicator
                
                # Insert code here ***

                # Parse the orthometric height 
                
                # Insert code here ***
                
                # Generate an error if the units of the orthometric height is not meters
                                   
                if gga_data[10].lower() != "m":
                    raise RuntimeError('Orthomeric height units are not meters!')                
                
                # Parse the geoid ellipsoid separation
                                   
               # Insert code here ***
                                   
                if gga_data[12].lower() != "m":
                    raise RuntimeError('Orthomeric height units are not meters!') 
                    
                # If there is more data then parse it
                
                if gga_data[13] != "":
                    self.corr_ages.append(float(gga_data[13]))
                    self.corr_stations.append(float(gga_data[14]))
                    
                # For now, ignore the checksum (this would become a computer science assignment
                
                # Make sure to update the previous gga time
                    
                # Insert code here ***

        # Set the reference ellipsoid to WGS84

        self.metadata["ellipsoid_name"] = "WGS84"
        self.metadata["geoid_name"] = "EGM08"
        self.metadata["height_relative_to"] = "geoid"

        # Let the user know how many GGA records there were in the file
        print("HYPACK RAW file contains: " + str(num_gga_recs) + " GGA records")

In [11]:
import import_ipynb
import os.path
import matplotlib as plt
from mycode.position import Position
from datetime import datetime, timezone

# get the absolute path to the current directory

abs_path=os.path.abspath(os.path.curdir)

# Instantiate a Position object and read data into it 

positions=Position()
positions.read_hypack_raw_file(abs_path+'/Data/000_1029.RAW')

Opening GNSS data file:/home/jupyter-semme/ESCI_872/Data/000_1029.RAW
HYPACK RAW Header start time and date: Tue Jul  3 10:29:07 2012
HYPACK RAW Header start time and date in UTC: Tue Jul  3 14:29:07 2012
HYPACK RAW file contains: 58 GGA records


Let us walk through the code:

You will notice that we read the code in three parts: 1) The header, 2) The data until the first GGA record, and 3) the data from the first GGA record

This seems cumbersome - the reason why has to do with the information contained within the GGA record

### 4.0.0 GGA Records

Research the trimble GGA record - is the epoch data complete (discuss on Piazza)

### 4.0.1 TND Records

Look at the TND record in the HYPACK file header (lines up to EOH) - can you use this data to complete the epochs (discuss on Piazza)

### 4.0.2 GGA and TND records

There appears to be a mismatch between the times in the GGA and the TND records - what is the problem? (discuss on Piazza)

### 4.0.3 Reconciling the Epochs

Analyze the code given to you - how does this deal with the issue? (discuss on Piazza)

### 4.0.4 Is the reconciliation complete?

Note that the first GGA record is received after the file header is created i.e., the time is different. Assuming that both the CPU time and the GGA time are correct, just relative to a different time zone, is there still the potential for getting the wrong date with the code as is provided here?

If so when do you think problems may occur?

### 4.0.5 Complete the HYPACK reading method 

Replace the sections marked **"# Insert code here** With actual code 

### 4.0.6 Create a map of the data read

Produce a map using the *position.draw()* method in the code cell below


<img align="left" width="5%" style="padding-right:10px;" src="Images/email.png">

*For issues or suggestions related to this notebook that should not be addressed on Piazza, write to: semmed@ccom.unh.edu*